# CA3 - Bayesian Network Text Classification
The project's aim is to catagorize the news into three different genres. To do so, **bayesian networks** are being applied in order to consider every word to contribute independently to the probability irrespective of the correlations. In this way, the computations will be lessened as well as having a reasonable final result.
## Libraries
> `time` library is being used to measure the time needed to reach the final network.
<br>
`operator` library is being used in the process of finding the final labels for test data.
<br>
As it is known, `numpy` and `pandas` libraries are two key factors in processing data. With using these tools, working with texts would be much more easier.
<br>
`nltk` plays a crucial role in this project. As it is needed to clean the input data, this library is a great choice to do so.

In [41]:
import time
import operator
import numpy as np
import pandas as pd
from nltk.corpus import stopwords 
from IPython.display import Markdown
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk import sent_tokenize, word_tokenize

## Input Data
>Two `.csv` files are given as the input data if this problem.
<br><br>
The first one is *data.csv* which is the the source file of this problem. To create a bayesian network, an input file is needed to extract the words to make a dictionary consisting of all the words which have been used before and their frequency as well. This input file contains many columns in which **short_description** and **header** were the useful ones as they have the most frequent words related to that specific subject. Additionally, **category** column is needed to keep the words in their own genres. Other columns of this input file are useless in this project.
<br><br>
The second file is *test.csv* which contain the same information as the first file, the only difference is that this file does not support any kind of data related to **category** of these news. The aim is to use the dictionary which have been built using the first file, and find the category of each news in the test file.

## Partitioning input data
>In classification tasks, it is important to test the final network to see whether it is working well or not. To do so, test data is needed which the system has not seen before. partitioning the data, hence, will do the same. According to this problem's data, separating 80 percent of the input for *training* and 20 percent for *validation* would be a reasonable choice. 
<br>
Note that to do so, we need to subtract 20 percent of each category for validation, and leave others as train data. If we just cut 20 percent of data and not from each category, the number of samples in train and test data may be unfair. For example, we may put most of a specific category in test data. In this way, the network will suffer from not having enough train data in that selected category.

## OverSampling
>Imbalanced learning is a result of imbalanced data. It means that the number of samples for each category is not equal to any other one. So this will lead the system to a point that many differences can be seen in Recall and Precision of different groups. 
<br>    
In order to tackle this issue, **oversampling** concept can be used. This concept suggest that for the catagories with less data, we can copy and repeat the samples we had before to make a dataset with equal number of examples in each category. In this way the imbalanced problem will be lessend, but *overfitting* may happen as the network is fitting to the input data more than its usual form. So, using this feature is kind of a trade-off and have to be controlled. 
<br><br>
In this problem, as many approaches has been applied before using oversampling, this feature will not add a significant amount of accuracy, recall, or precision to the model.

In [42]:
def partition_data(data):

    data = data.sample(frac=1)
    grouped_data = data.groupby(["category"])
    train_prerequisits = []
    test_prerequisits = []
    max_size = data['category'].value_counts().max()
    
    for index, value in enumerate(data["category"].unique()):
        selected_group = grouped_data.get_group(value)
        partition = int(0.8 * len(selected_group))
        train_prerequisits.append(selected_group[0:partition])
        test_prerequisits.append(selected_group[partition: ])

    train_prerequisits_oversampled = oversample(train_prerequisits, max_size)
    train_data = pd.concat([train_prerequisits_oversampled[i] for i in range(len(train_prerequisits_oversampled))])

    test_data = pd.concat([test_prerequisits[i] for i in range(len(test_prerequisits))])

    return train_data, test_data

def oversample(data, max_size):
    for index, value in enumerate(data):
        value.append(value.sample(max_size - len(value), replace=True), ignore_index = True)
    return data

## Bayesian Rules

> This approach tries to classify each category using the formula explained below:
> $$ P(category\ |\ word) = \frac{P(word\ |\ category)P(category)}{P(word)} $$
>
>
> #### $P(category\ |\ word)$
>> As it was explained in before, we are looking for building a bayesin classifier. In order to do so, we need to have the probability of each category for every instance. In this way, the one with the biggest probability, will be the detected category. So, the key to this problem is to find $P(category\ |\ sample)$ which is being obtained using $P(category\ |\ word)$ for each word in sample. This probability is Posterior Probability.
>>
>
> #### $P(word\ |\ category)$
>> According to bayes rules, as the posterior probability is not computable, we can obtain its probability using 
$P(word\ |\ category)$. This probability, called Likelihood, represents the probbility of having each word in a specific category. To calculate this parameter, a dictionary is being created in which every word and its frequency in all the categories have been calculated before.
>>
>
> #### $P(category)$ 
>> This refers to the probability of the selected category. Each category has a chance equal to $\frac{number\ of\ samples\ in\ category\ x}{number\ of\ all\ samples} $. This probability is called Class Prior Probability.
>>
>
> #### $P(word)$
>> This parameter, known as Predictor Prior Probability, represents the total probability of having a word in train data. Since it is similar in all cases that are being examined, this part is usually being ommited.
>>
> 

## The Solver
To create a network, a python class is needed to process train data and create a dictionary in which we keep the words and their probability for each different category. 
<br>
> #### 1. `prepare_dictionaries()`
As this problem is being solved using a bayesian network, a dictionary for each category is needed in which the words and their frequency is being recorded. In order to do, first we initialize a dictionary for each category and then, create a key for each word that is being seen in that specific category. The value of each key refers to its frequency in the selected category. Using this 2D dictionary(one dimension for different categories and the other for words) the probability of each word is calculable.
<br>
It should be noted that in this stage, a set of all the words in all the descriptions will be created as well. This set will be used in further computations.
> #### 2. `convert_description_to_words()`
In order to obtain enough information from input data, it is cruical to tokenize and clean the input data. This function has the aim of doing so. <br>
Firstly, a tokenizer will be initialized using `RegexTokenizer` function. After extract the words, it is needed to do a preprocess on them. In this part, we will eliminate stop words which are repetitious and are being used in almost every sentence by comparing words with a set of stopwords obtained from `nltk` library. Next, all the capital alphabets should be converted to small characters in order to be equal with the same word but different in capital alphabets. Finally, it is important to find the origin of each word. For example, *starts* and *start* are not different from each other. To prevent this from happening, a tool called **lemmatizer** from `nltk` library is being used. This will convert each word to its origin and will help us to have a same key for all of similar words. The list of words will be returned after processing them in this way.
> #### 3. `num_of_words()`
This function simply counts the number of words in a selected category. The result will be needed in predicting labels further.
> #### 4. `find_labels()`
This function is being used in order to predict the labels of a group of test data. To do so, **Bayesian Rules** should be applied. 
<br> <br>
$$P(category\ |\ description) = \frac{P(description\ |\ category)P(category)}{P(description)} $$
<br>
$$P(description\ |\ category) = \prod\limits_{for\ word\ in\ ala\ words\ of\ a\ description} P(word\ |\ category) $$
According to bayes rule, we can calculate the probability of each category using these formulas. This function is computing the probability of each category for each data and chooses the one which has the biggest value as the predicted category. <br>
Note that in each step of computation, it is better to multiply the probabilities with a bug number in order to prevent from a number to lead to 0.
> #### 5. `apply_on_test()`
This function simply tries to predict the labels of the *test.csv* file. First, the test file will be read and then, applied `find_labels()` function on it.

In [43]:
class Solver:
    def __init__(self, train_data):
        self.train_data = train_data.reset_index(drop=True)
        self.words, self.dictionaries = self.prepare_dictionaries()
    
    def prepare_dictionaries(self):
        all_words = set()
        train_dict = dict()
        
        for index, value in enumerate(self.train_data["category"].unique()):
            train_dict[value] = dict()
        
        for index, value in enumerate(self.train_data['short_description']):
            list_of_words = self.convert_description_to_words(value)
            for i, word in enumerate(list_of_words):
                all_words.add(word)
                selected_catagory = self.train_data["category"][index]
                if word in train_dict[selected_catagory]:
                    train_dict[selected_catagory][word] += 1
                else:
                    train_dict[selected_catagory][word] = 1

        return all_words, train_dict
    
    def convert_description_to_words(self, value):
        lemmatizer = WordNetLemmatizer()
        tokenizer = RegexpTokenizer(r'\w+')
        stop_words = set(stopwords.words('english'))

        final_words=[]
        list_of_words = tokenizer.tokenize(value)

        for i, value in enumerate(list_of_words):
            if value in stop_words:
                continue
            word = value.lower()
            final_words.append(lemmatizer.lemmatize(word))
            
        return final_words
    
    def num_of_words(self, category):
        ans = 0
        for index, (key, value) in enumerate(self.dictionaries[category].items()):
            ans += value
        return ans
    
    def find_labels(self, data):
        labels = []
        prior = dict()
        num_of_words = dict()
        grouped_data = self.train_data.groupby(["category"])

        
        for index, value in enumerate(self.train_data["category"].unique()):
            num_of_words[value] = self.num_of_words(value)
            prior[value] = len(grouped_data.get_group(value)) / len(self.train_data)

        for index, value in enumerate(data['short_description']):
            p = dict()
            all_words = self.convert_description_to_words(value)
            
            for i, category in enumerate(self.train_data["category"].unique()):
                p[category] = prior[category]
                
                for index, word in enumerate(all_words):
                    if word in self.dictionaries[category]:
                        p[category] *= (self.dictionaries[category][word] + 1)
                    p[category] *= 20000 / (len(self.words) + num_of_words[category])

            labels.append(max(p.items(), key=operator.itemgetter(1))[0])
        
        return labels
    
    def apply_on_test(self, test_file):
        
        test_data = pd.read_csv(test_file)
        test_data = test_data.replace(np.nan, '', regex=True)
        test_data['short_description'] = test_data['short_description'] + " " + test_data['headline']
        test_data = pd.DataFrame(test_data['short_description'])

        return self.find_labels(test_data)   

## Precision, Recall, Accuracy, and Confusion Matrix
As it was said in the project, It is cruical to ensure that this network is precise enough. To measure this, some standards have been defined.
> **1. Confusion Matrix**
> A confusion matrix is a way of showing prediction results briefly on a classification problem(in this case Bayesian Network). The number of correct and incorrect label predictions are summarized with count values and broken into each class. In this way, we can see for each label how many predictions were correct and what was the type of our mistakes and the number of them as well. From observing Confusion Matix, accuracy, precision and recall can be calculated easily as this matrix have the summery of all the information needed to examine a classification method. <br><br>
> **2. Accuracy**
> Accuracy is the ratio of the total number of correct detections divide to total number of predictions. This measure examines the system generally.<br>
$$Accuracy = \frac{Number\ of\ correct\ detections}{Number\ of\ all\ predictions}$$
> **3. Precision**
> Precision can be defined as the ratio of the total number of correctly classified the specified label divide to total number of predictions with that specific label. In fact, this measure shows that how much predicting a specific label is accurate. High precision indicates the probability which an examples' label is(if this category has been selected). <br><br>
$$Precision = \frac{Number\ of\ correct\ detections\ for\ a\ specific\ category}{Number\ of\ all\ predictions\ with\ this\ specific\ category}$$
<br><br>
> **4. Recall**
> Recall can be defined as the ratio of the total number of correctly classified the specified label divide to the total number of examples with that selected label. Actually, high recall rate shows that the category is being recognized right in most of the cases. <br>
$$Recall = \frac{Number\ of\ correct\ detections\ for\ a\ specific\ category}{Number\ of\ all\ examples\ with\ this\ specific\ category}$$

In [44]:
def precision(index, confusion_matrix):
    col = confusion_matrix[:, index]
    return confusion_matrix[index, index] / col.sum()
    
def recall(index, confusion_matrix):
    row = confusion_matrix[index, :]
    return confusion_matrix[index, index] / row.sum()

def accuracy(confusion_matrix):
    return confusion_matrix.trace() / confusion_matrix.sum() 

def calc_confusion_matrix(actual, predicted):
    labels = pd.DataFrame(list(zip(actual, predicted)), columns =['Actual', 'Predicted'])
    size = len(labels['Actual'].unique())
    confusion_matrix = np.zeros((size, size))
    
    dict_label = dict()
    for index, value in enumerate(labels['Actual'].unique()):
        dict_label[value] = index
    
    for i in range(len(labels['Actual'])):
        confusion_matrix[dict_label[labels['Actual'][i]]][dict_label[labels['Predicted'][i]]] += 1
        
    return confusion_matrix, dict_label

def print_result(category, index, confusion_matrix):
    display(Markdown("### The result for " + category.lower() + " category is shown below:\n"))
    display("Percision: %s" % precision(index, confusion_matrix))
    display("Recall: %s" % recall(index, confusion_matrix))
    
def print_confusion_matrix(dict_label, confusion_matrix):
    table = "|Category|"
    for i, (key, value) in enumerate(dict_label.items()):
        table += key
        table += "|"
    table += "\n|---"
    for i in range(len(confusion_matrix[0])):
        table += "|---"
    table += "|\n"
    for i, (key1, value1) in enumerate(dict_label.items()):
        table += "|"
        table += key1
        table += "|"
        for i, (key2, value2) in enumerate(dict_label.items()):
            table += (confusion_matrix[value1][value2]).astype('str')
            table += "|"
        table += "\n"
    table += "<caption  style=\"text-align:center\">Confusion Matrix</caption>" 
    display(Markdown(table))

## Initial steps to clean input data
> Firstly, the input data will be converted to a pandas data structure using `read_csv()` function from `pandas` library. 
<br>
One of the first problems which is being faced is that some of columns have **Nan** value. To prevent errors, all the **Nan** values will be replaced with a space(' ' character) `replace()` function. After all, We need to keep the data in **short_description**, **header** and **category** columns and the rest will not be used. To do so, two measures has to be taken. 
<br>
Firstly, as this method works with bag of words and does not care about the sentences, we can combine **header** and **short_description** columns into a single column. All the data needed to create a dictionary will be kept in **short_description** column. Lastly, the needed data will consist of the new **short_description** column and **category** column, and the rest of data will be thrown away.

In [45]:
data = pd.read_csv("Attachment/data.csv")
data = data.replace(np.nan, '', regex=True)
data['short_description'] = data['short_description'] + " " + data['headline']
data = pd.DataFrame(data = {'category' : data.loc[:, 'category'], 'short_description' : data.loc[:, 'short_description']})

## Phase One
> In this part of the problem, we are about to predict labels for the input data. The aim is to check the network on only two different categories. In order to do so, data is being categorized using `groupby()` and then, by applying `get_group()` function, the two proposed groups are being extracted.
<br>
After all, by creating an instance of the solver class which has been declared before in the report, and the function `find_labels()`, the labels will be predicted. Finally, by the functions which has been defined to find the effectiveness of the network, the total accuracy, recall and precision will be calculated. 

In [46]:
grouped_data = data.groupby(["category"])
phase_one_data = pd.concat([grouped_data.get_group('TRAVEL'), grouped_data.get_group('BUSINESS')])
train_data_phase_one, test_data_phase_one = partition_data(phase_one_data)
solve_phase_one = Solver(train_data_phase_one)

In [48]:
prediction = solve_phase_one.find_labels(test_data_phase_one)
confusion_matrix, dict_label = calc_confusion_matrix(test_data_phase_one['category'], prediction)
display(Markdown("### The confusion matrix is shown below: "))
print_confusion_matrix(dict_label, confusion_matrix)

display(Markdown("### Total accuracy is: " + str(accuracy(confusion_matrix))))
for index, value in enumerate(test_data_phase_one['category'].unique()):
    print_result(value, dict_label[value], confusion_matrix)

### The confusion matrix is shown below: 

|Category|TRAVEL|BUSINESS|
|---|---|---|
|TRAVEL|1716.0|64.0|
|BUSINESS|90.0|979.0|
<caption  style="text-align:center">Confusion Matrix</caption>

### Total accuracy is: 0.9459459459459459

### The result for travel category is shown below:


'Percision: 0.9501661129568106'

'Recall: 0.9640449438202248'

### The result for business category is shown below:


'Percision: 0.9386385426653883'

'Recall: 0.9158091674462114'

| Phase One | Travel | Business | 
| --- | --- | --- | 
| Recall | 0.9640 | 0.9158 | 
| Precision | 0.9502 | 0.9386 | 
| Accuracy(Total) | | 0.9459 | 
<caption  style="text-align:center">Result for testing phase one of the project</caption>


## Phase Two
> In this part of the problem, we are about to predict labels for the input data. The difference is that we want to check the network on all different categories in this stage. In order to do so, data is being categorized using `groupby()` and then, by applying `get_group()` function, the two proposed groups are being extracted.
<br>
After all, by creating an instance of the solver class which has been declared before in the report, and the function `find_labels()`, the labels will be predicted. Finally, by the functions which has been defined to find the effectiveness of the network, the total accuracy, recall and precision will be calculated. 

In [49]:
train_data_phase_two, test_data_phase_two = partition_data(data)
solve_phase_two = Solver(train_data_phase_two)

In [50]:
prediction = solve_phase_two.find_labels(test_data_phase_two)
confusion_matrix, dict_label = calc_confusion_matrix(test_data_phase_two['category'], prediction)
display(Markdown("### The confusion matrix is shown below: "))
print_confusion_matrix(dict_label, confusion_matrix)

display(Markdown("### Total accuracy is: " + str(accuracy(confusion_matrix))))
for index, value in enumerate(test_data_phase_two['category'].unique()):
    print_result(value, dict_label[value], confusion_matrix)

### The confusion matrix is shown below: 

|Category|BUSINESS|STYLE & BEAUTY|TRAVEL|
|---|---|---|---|
|BUSINESS|941.0|26.0|102.0|
|STYLE & BEAUTY|70.0|1598.0|69.0|
|TRAVEL|67.0|31.0|1682.0|
<caption  style="text-align:center">Confusion Matrix</caption>

### Total accuracy is: 0.920409943305713

### The result for business category is shown below:


'Percision: 0.87291280148423'

...

'Percision: 0.965558912386707'

'Recall: 0.9199769717904432'

### The result for travel category is shown below:


'Percision: 0.9077172153264975'

'Recall: 0.9449438202247191'

| Phase Two | Travel | Business | Style & Beauty |
| --- | --- | --- | --- |
| Recall | 0.9449 | 0.8802 | 0.9199 |
| Precision | 0.9077 | 0.8729 | 0.9656 |
| Accuracy(Total) | | 0.9204 |
<caption  style="text-align:center">Result for testing phase two of the project</caption>


## Test the `test.csv` file
Finally, the network should be applied on a test data which does not have specific categories. To do so, `apply_on_test()` function from solver class should be used. This function has been declared and explained before in this project. After finding the labels, they are being saved using `to_csv()` function and create a *csv* flie named *output.csv*.

In [40]:
start_time = time.time()

ans = solve_phase_two.apply_on_test("Attachment/test.csv")
answer = pd.DataFrame(list(zip([i for i in range(len(ans))], ans)), columns =['index', 'category'])
answer.to_csv ('output.csv', index = False, header=True)

total_time = time.time() - start_time
display(Markdown("It takes %s seconds to predict the labels of test data." %total_time))

It takes 2.5437231063842773 seconds to predict the labels of test data.

# Questions and Explanations

**1. Stemming or Lemmatization**
> The application of both stemming and lemmatization is to produce morphological variants of a root/base word with the aim of reducing inflectional forms and sometimes derivationally related forms.
> **stemming:** Stems are created by removing the suffixes or prefixes used with a word. Stemming a word or sentence may result in creating words that dows not really exist. For example, the word *troubling* will change into *troubl* and not the right form(*trouble*) <br>
> **Lemmatization:** Lemmatization, unlike Stemming, reduces the inflected words while ensuring that the base word belongs to the language.
<br>
>> According to the definitions of these two approaches, obviously it is better to use Lemmatization inasmuch as it reachs the real origin of each word. As an instance, the word *'studies'* will be *'studi'* using Stemming but if Lemmatization is being pplied, the word will change into its source, *'study'*.

**2. TF-IDF**
> TF-IDF, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect the importance of each document in a collection. It is often used as a weighting factor in searches of information retrieval, text mining, and calssification as well.
<br>
>>**TF(Term Frequency):** TF is actually calculating the frequency of a word in a document. The thought behind it is that if a word occurs multiple times in a document, its relevance should be more as it should be more meaningful than other words that appear fewer times.
<br>
$$ TF(word,documanet) = \frac{count\ of\ word\ in\ documanet}{number\ of all\ words\ in\ documanet} $$
<br>
>>**IDF(Inverse Document Frequency):** If a word occurs many times in a document and also along many other documents, it may be because of its general meaning; not because it was relevant or meaningful. In other words, IDF measures the rank of the specific word for its relevancy within the text and not its frequency. Stop words which contain unnecessary information such as “a”, “into” and “and” are a good example of less important words as they are being used generally.
<br>
$$ IDF(word) = \frac{number\ of\ all\ ducuments}{occurrence\ of\ word\ in\ documents+1} $$
>>In the furmula, the denominator is incremented in order to prevent the problem of not having a word in all the documents at all.
>><br>
**Revelance:** All in all, the total TF-IDF weight for a token in a document is:
$$ TF-IDF = TF \times IDF $$
>>
> **How to use this measurement in Bayesian Networks?** <br><br>
>As it was said before, the probability for each word being in a certain category is being calculated using ites frequency in the selected documents. 
<br>
This can easily change into TF-IDF measure as this approach is returning a value based on the importance of words. After doing so, the probability will be calculated in the way shown below:
<br>
$$ P(word\ |\ category) = \frac{TF-IDF\ value\ of\ the\ word\ in\ category\ + 1}{Sum\ of\ all\ the\ TF-IDF\ values\ in\ category\ + number\ of\ all\ words} $$

**3. High Precision**
>If we just pay attention to the precision and not recall, we won't recognize that how often the system even recognizes an example which its real category is the one we are looking for. Consider a system that diagnoses corona virus in people. If the system has low recall and high precision, if the result is true, most of the time the patient's test was positive as well, but the problem is that the system cannot recognize many of them due to its low recall. As a result there will be lots of patients who got a negative result even though they had the corona virus. Hence, there will be plenty of problems as a result of low recall in this system.

**4. New words**
>As we know, if a word is not in the dictionary of a category, the probability of that specific word will be 0 result in a probability of 0 for that category, which will not be true most of the times. To prevent this from happening, here's a solution:
<br>
$$ P(word\ |\ category) = \frac{frequency\ of\ the\ word\ in\ category\ + 1}{number\ of\ words\ in\ category\ + number\ of\ all\ words} $$
>>In this way, we are actually assuming that each word is being repeated at least one time in each category. Hence, this will give a little value to the probability that was detected as zero before, and will not ruin the whole probability realted to a specific category.
<br><br>
>>> - The Answer to the Question: <br> It has been obtained that if a word is repeated just in one category, without this method, the only category that has a value except zero is the one which has the word actually. In other words, the rest of the sentence will have no importance dur to the zero probability.